<a href="https://colab.research.google.com/github/shitote/dist-repo/blob/main/transfer_learning_scaling_up.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Rranfer Learnig Scaling up (🍰👁 Food Vision)

In [ ]:
!nvidia-smi

In [ ]:
!wget https://raw.githubusercontent.com/mrdbourke/tensorflow-deep-learning/main/extras/helper_functions.py

In [43]:
from helper_functions import create_tensorboard_callback, plot_loss_curves, unzip_data, compare_historys, walk_through_dir

## 101 Working with less data

In [ ]:
!wget https://storage.googleapis.com/ztm_tf_course/food_vision/101_food_classes_10_percent.zip
unzip_data("101_food_classes_10_percent.zip")

In [45]:
train_dir = "101_food_classes_10_percent/train/"
test_dir = "101_food_classes_10_percent/test/"

In [ ]:
# image classes
walk_through_dir("101_food_classes_10_percent")

In [ ]:
# Se up data inputs
import tensorflow as tf
IMG_SIZE = (224, 224)
train_data_all_10_percent = tf.keras.preprocessing.image_dataset_from_directory(train_dir,
                                                                               label_mode="categorical",
                                                                               image_size=IMG_SIZE)
test_data = tf.keras.preprocessing.image_dataset_from_directory(test_dir,
                                                                label_mode='categorical',
                                                                image_size=IMG_SIZE,
                                                                shuffle=False)

## Train the model.
steps to take.
* Create a modelCheckpoint callback
* Create a data augmentation.
* Build a headlsess functional api model.
* compile the model.
* Feature extraxt


In [48]:
checkpoint_path = "101_classes_10_percent_data_model_checkpoint"
checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(checkpoint_path,
                                                         save_weights_only=True,
                                                         monitor="val_accuracy",
                                                         save_best_only=True)

In [49]:
# Create data augmentation layer
import tensorflow as tf
from tensorflow.keras import layers
from tensorflow.keras.layers.experimental import preprocessing
from tensorflow.keras.models import Sequential

data_augmentaion = Sequential([
    preprocessing.RandomFlip("horizontal"),
    preprocessing.RandomRotation(0.2),
    preprocessing.RandomHeight(0.2),
    preprocessing.RandomWidth(0.2),
    preprocessing.RandomZoom(0.2)
], name="dara_augmentation")


In [50]:
# set up the base model.
base_model = tf.keras.applications.EfficientNetB0(include_top=False)
base_model.trainable = False

# Setup model architecture with trainabe top layers
inputs = layers.Input(shape=(224, 224, 3), name='input_layer')
x = data_augmentaion(inputs)
x = base_model(x, training=False)
x = layers.GlobalAveragePooling2D(name='gloabal_avg_pool_layer')(x)
outputs = layers.Dense(len(train_data_all_10_percent.class_names), activation="softmax", name="output_layer")(x)
model = tf.keras.Model(inputs, outputs)



In [ ]:
model.compile(loss='categorical_crossentropy',
              optimizer=tf.keras.optimizers.Adam(),
              metrics=["accuracy"])

# Fit the model
history_all_classes_10_percent = model.fit(train_data_all_10_percent,
                                           epochs=5,
                                           validation_data=test_data,
                                           validation_steps=int(0.15 * len(test_data)),
                                           callbacks=[checkpoint_callback])

In [ ]:
fine_tuning_results = model.evaluate(test_data)
fine_tuning_results

In [ ]:
plot_loss_curves(history_all_classes_10_percent)

## FIne tuning

In [54]:
# Unfreeze all the layers in the base model to be trainable
base_model.trainable = True

# Refreeze every layer excep the last 5
for layer in base_model.layers[:-5]:
  layer.trainable = False

In [55]:
model.compile(loss='categorical_crossentropy',
              optimizer=tf.keras.optimizers.Adam(0.0001),
              metrics=['accuracy'])

In [ ]:
for layer in model.layers:
  print(layer.name, layer.trainable)

In [ ]:
for layer_number, layer in enumerate(model.layers[2].layers):
  print(layer_number, layer.name, layer.trainable)


In [ ]:
# Fien tune for 5 more epoches
fine_tune_epochs = 10

# Fine-tune the model.
history_all_classes_10_percent_fine_tune = model.fit(train_data_all_10_percent,
                                                     epochs=fine_tune_epochs,
                                                     validation_data=test_data,
                                                     validation_steps=int(0.15 * len(test_data)),
                                                     initial_epoch=history_all_classes_10_percent.epoch[-1])

In [ ]:
history_all_classes_10_percent_fine_tune_results = model.evaluate(test_data)

In [ ]:
compare_historys(history_all_classes_10_percent,
    history_all_classes_10_percent_fine_tune)

## Saving and loading the model
To use the model is an external application, we'll need to save it export it somewhare else.


In [61]:
model.save("drive/MyDrive/tensorflow/101_food_classes_10_percent_saved_big_dog_model")

In [62]:
import tensorflow as tf
loaded_model = tf.keras.models.load_model("drive/MyDrive/tensorflow/101_food_classes_10_percent_saved_big_dog_model")

In [ ]:
loaded_model_results = loaded_model.evaluate(test_data)

In [ ]:
history_all_classes_10_percent_fine_tune_results

## Evaluating the performance of the model across all different classes

In [ ]:
import tensorflow as tf

!wget https://storage.googleapis.com/ztm_tf_course/food_vision/06_101_food_class_10_percent_saved_big_dog_model.zip

In [66]:
unzip_data("/content/06_101_food_class_10_percent_saved_big_dog_model.zip")

In [ ]:
model = tf.keras.models.load_model("/content/06_101_food_class_10_percent_saved_big_dog_model")

In [ ]:
# Evaluate the loaded model on test data
results_downloaded_model = model.evaluate(test_data)


In [ ]:
results_downloaded_model

## Making predictions.


In [71]:
from re import VERBOSE
# make predictions with the model.
preds_probs = model.predict(test_data, verbose=1)
preds_probs

790/790 [==============================] - 61s 77ms/step


array([[5.95422499e-02, 3.57421641e-06, 4.13771868e-02, ...,
        1.41387257e-09, 8.35307801e-05, 3.08975508e-03],
       [9.64016914e-01, 1.37533007e-09, 8.47806223e-04, ...,
        5.42869529e-05, 7.83620738e-12, 9.84662241e-10],
       [9.59258676e-01, 3.25337387e-05, 1.48669758e-03, ...,
        7.18913384e-07, 5.43975261e-07, 4.02759979e-05],
       ...,
       [1.51387385e-05, 4.09730448e-04, 8.02507671e-10, ...,
        2.17429133e-05, 1.07972855e-05, 5.37898600e-01],
       [5.93182491e-03, 4.92365798e-03, 9.98234283e-03, ...,
        1.19894612e-04, 1.68902443e-05, 4.52180393e-02],
       [3.13634686e-02, 7.50524970e-03, 4.29750886e-04, ...,
        5.03464777e-04, 5.20560252e-06, 6.90628529e-01]], dtype=float32)